In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
# Read the dataset
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
application_df = pd.read_csv(url)
# Display the first few rows of the dataset
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the 'EIN' and 'NAME' columns
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)


In [5]:
# Determine the number of unique values in each column
unique_counts = application_df.nunique()

print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [7]:
# Choose a cutoff value
cutoff = 500

# Create a list of application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff].index.tolist()

# Replace in the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [8]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [9]:
# Look at CLASSIFICATION value counts > 1
classification_counts_above_1 = classification_counts[classification_counts > 1]
print(classification_counts_above_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [10]:
# Choose a cutoff value
cutoff = 1000

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace in the dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [11]:
# Convert categorical data to numeric with pd.get_dummies
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION'])

# Display the updated DataFrame
application_df.head()

,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,...,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other
0,Independent,ProductDev,Association,1,0,N,5000,1,0,1,...,0,0,0,0,1,0,0,0,0,0
1,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,CompanySponsored,ProductDev,Association,1,0,N,5000,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [12]:
# Split our preprocessed data into features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Display the shape of the arrays
print("Features shape:", X.shape)
print("Target shape:", y.shape)


from sklearn.model_selection import train_test_split

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


Features shape: (34299, 22)
Target shape: (34299,)
X_train shape: (27439, 22)
X_test shape: (6860, 22)
y_train shape: (27439,)
y_test shape: (6860,)


In [16]:
# Apply one-hot encoding to categorical columns
columns_to_encode = ['AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT']
X_train_encoded = pd.get_dummies(X_train, columns=columns_to_encode)
X_test_encoded = pd.get_dummies(X_test, columns=columns_to_encode)

# Make sure both datasets have the same columns
all_columns = set(X_train_encoded.columns).union(set(X_test_encoded.columns))
missing_columns = all_columns - set(X_test_encoded.columns)
for col in missing_columns:
    X_test_encoded[col] = 0  # Add missing columns to testing data with default values

# Reorganize columns in the testing dataset to match the order in the training dataset
X_test_encoded = X_test_encoded[X_train_encoded.columns]

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler on the encoded training data
X_scaler = scaler.fit(X_train_encoded)

# Scale the encoded data
X_train_scaled = X_scaler.transform(X_train_encoded)
X_test_scaled = X_scaler.transform(X_test_encoded)

In [17]:
# Define the model
nn = tf.keras.models.Sequential()

# Number of input features
input_features = X_train_scaled.shape[1]

In [18]:
# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

In [19]:
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [20]:
# Display model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5632      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16,001
Trainable params: 16,001
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Train the model
model_history = nn.fit(X_train_scaled, y_train, epochs=150, verbose=2, validation_split=0.2)


Epoch 1/150
686/686 - 3s - loss: 0.5697 - accuracy: 0.7198 - val_loss: 0.5565 - val_accuracy: 0.7365 - 3s/epoch - 5ms/step
Epoch 2/150
686/686 - 2s - loss: 0.5559 - accuracy: 0.7271 - val_loss: 0.5480 - val_accuracy: 0.7380 - 2s/epoch - 2ms/step
Epoch 3/150
686/686 - 1s - loss: 0.5524 - accuracy: 0.7284 - val_loss: 0.5528 - val_accuracy: 0.7363 - 1s/epoch - 2ms/step
Epoch 4/150
686/686 - 1s - loss: 0.5506 - accuracy: 0.7312 - val_loss: 0.5462 - val_accuracy: 0.7374 - 1s/epoch - 2ms/step
Epoch 5/150
686/686 - 1s - loss: 0.5494 - accuracy: 0.7309 - val_loss: 0.5442 - val_accuracy: 0.7374 - 1s/epoch - 2ms/step
Epoch 6/150
686/686 - 2s - loss: 0.5485 - accuracy: 0.7315 - val_loss: 0.5439 - val_accuracy: 0.7382 - 2s/epoch - 3ms/step
Epoch 7/150
686/686 - 2s - loss: 0.5482 - accuracy: 0.7331 - val_loss: 0.5470 - val_accuracy: 0.7358 - 2s/epoch - 3ms/step
Epoch 8/150
686/686 - 1s - loss: 0.5476 - accuracy: 0.7316 - val_loss: 0.5468 - val_accuracy: 0.7376 - 1s/epoch - 2ms/step
Epoch 9/150
686/

In [22]:
# Evaluate model on test data
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

215/215 - 0s - loss: 0.5849 - accuracy: 0.7255 - 421ms/epoch - 2ms/step
Test Loss: 0.5849151015281677
Test Accuracy: 0.7255101799964905


In [23]:
# Export the model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")